<a href="https://colab.research.google.com/github/achett/IB-Generation/blob/main/IB_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# !pip -q install openai==0.27.0 langchain huggingface_hub
# !pip install --upgrade --quiet  openllm
# !pip install gpt4all
# !pip install langchain_experimental
# !pip install llama-cpp-python

!pip install rouge
!pip install bert-score
!pip install sumy
!pip install langchain
!pip install openai==0.27.0
!pip install python-dotenv
!pip install xmltodict

In [9]:
import os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain, LLMSummarizationCheckerChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GPT4All
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.pubmed.tool import PubmedQueryRun
import openai
import textwrap
import dotenv
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
# from langchain_experimental.chat_models import Llama2Chat
from os.path import expanduser
from langchain_community.llms import LlamaCpp
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
import xmltodict

In [10]:
######################
# AZURE OPENAI MODEL
######################
dotenv.load_dotenv('/content/drive/MyDrive/Colab Notebooks/IB Generation/.env')

AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "chatgpt" #GPT turbo
TEMPERATURE = float(os.environ.get("TEMPERATURE"))
if 'AZURE_OPENAI_KEY' not in os.environ:
    raise RuntimeError("No 'AZURE_OPENAI_KEY' env var set.  Please see README.")
if 'AZURE_OPENAI_VERSION' not in os.environ:
    raise RuntimeError("No API version specified, please set it up as an environment variable")

# configure azure openai for langchain and/or llm
openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = AZURE_OPENAI_VERSION # this may change in the future
                                    # for langchain, set this version in environment variables using OPENAI_API_VERSION


llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=0, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [ ]:
# ######################
# # LLAMA
# ######################
# model_path = expanduser("/content/drive/MyDrive/Colab Notebooks/IB Generation/models/llama-2-7b-chat.Q4_0.gguf")

# llm = LlamaCpp(
#     model_path=model_path,
#     temperature=0.0,
#     streaming=False,
#     max_tokens=2000
# )
# llm = Llama2Chat(llm=llm)

# template_messages = [
#     SystemMessage(content="Generate a summary of the following text:"),
#     HumanMessagePromptTemplate.from_template("{text}"),
# ]

# prompt_template = ChatPromptTemplate.from_messages(template_messages)


# chain = LLMChain(llm=llm, prompt=prompt_template)

# print(
#     chain.run(
#         text=content
#     )
# )

In [11]:
######################
# DATA
######################
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source.txt', 'r') as file:
    content = file.read()

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target.txt', 'r') as file:
    target = file.read()

content = ' '.join(content.split())
target = ' '.join(target.split())

wrapped_text = textwrap.fill(content,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100,
and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its
toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the
reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period.
Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was
sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and
100 mg/kg groups, no toxicologically significant changes were noted in any examination during the
dosing period. In the 800 mg/kg group, vomiting was observed in males and females on 1 to 5 days
mainly at Weeks 1 and 2 of dosing. Salivation was observed in males and females immediately after
dosing during the dosing period. Decreased body weight was noted in males and females during the
dosing period. Low eryt

In [13]:
######################
# METRICS
######################
from rouge import Rouge
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer

def calculate_rouge2_f1(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    rouge_2_f1 = scores[0]['rouge-2']['f']
    return rouge_2_f1

def calculate_bertscore_idf(reference, hypothesis):
    # Using roberta-large model as an example, you can choose other models
    scorer = BERTScorer(model_type='bert-base-uncased')
    P, R, F1 = scorer.score([hypothesis], [reference])
    return F1.item()

In [14]:
########################
# EXTRACTIVE - SUMY: LexRank, Luhn, LSA, TextRank
########################
import re
import nltk
nltk.download('punkt')
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer

def count_sentences_and_calculate(text, ratio):
    # Count the number of sentences by splitting based on common punctuation
    sentences = re.split(r'[.!?]+', text)
    # Filter out any empty strings that may result from the split
    sentences = [sentence for sentence in sentences if sentence.strip()]
    sentence_count = len(sentences)

    # Calculate ratio of the sentence count and round to the nearest whole number
    result = round(sentence_count * ratio)

    return result


# Summarizer types
# summarizer = LexRankSummarizer()
# summarizer = LuhnSummarizer()
# summarizer = LsaSummarizer()
summarizer = TextRankSummarizer()

# Set number of sentences to be in summary
summary_ratio = 0.4
sumsent_n = count_sentences_and_calculate(content, summary_ratio)

# Parser
parser = PlaintextParser.from_string(content, Tokenizer("english"))

# Create summary
sum2 = summarizer(parser.document, sumsent_n)

# Combine sentences into one summary
sum_ex = ''
for sentence in sum2:
    sum_ex=sum_ex + str(sentence)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_ex)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_ex)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_ex)/len(content)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

ROUGE-2 F1 Score: 0.49659863446156693
BERTScore F1 (IDF) Score: 0.8015055656433105
Length Ratio: 0.5091191307722157


In [15]:
######################
# ABSTRACTIVE + ZERO SHOT PROMPT
######################
llm_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a summary of the following text:"),
        ("user", "{prompt}")
    ])

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

sum_abz = chain.invoke({"prompt":content})

wrapped = textwrap.fill(sum_abz,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abz)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_abz)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abz)/len(content)}")
print(wrapped)

ROUGE-2 F1 Score: 0.14556961530443857
BERTScore F1 (IDF) Score: 0.714145839214325
Length Ratio: 0.4823438106325184
Project 1 was tested for toxicity in cynomolgus monkeys over a 4-week period with doses ranging from
0 (control) to 800 mg/kg, administered daily. Each dose group consisted of 3 males and 3 females,
with an additional 3 males and 3 females in the 800 mg/kg group to assess toxicity reversibility
during a 4-week recovery period. The control group received a 0.5 w/v% methylcellulose solution. No
deaths or sacrifices due to moribundity occurred. No significant toxicological changes were observed
in the 10, 30, and 100 mg/kg groups. However, in the 800 mg/kg group, symptoms such as vomiting,
salivation, decreased body weight, low erythrocyte count, hematocrit value, hemoglobin
concentration, high triglycerides, and glucose levels, as well as changes in liver and adrenal
weights, were noted. Toxicokinetic analysis showed that the maximum concentration (Cmax) and area
under the c

In [16]:
######################
# Abstractive + Few Shot Prompt
######################
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source2.txt', 'r', errors='ignore') as file:
    source2 = file.read()
source2 = ' '.join(source2.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source3.txt', 'r') as file:
    source3 = file.read()
source3 = ' '.join(source3.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target2.txt', 'r') as file:
    target2 = file.read()
target2 = ' '.join(target2.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target3.txt', 'r') as file:
    target3 = file.read()
target3 = ' '.join(target3.split())

examples = [
    {"input": source2, "output": target2},
    {"input": source3, "output": target3},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

llm_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Generate a summary of the following text:"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)


output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

sum_abf = chain.invoke({"input":content})

wrapped = textwrap.fill(sum_abf,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abf)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_abf)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abf)/len(content)}")
print(wrapped)

ROUGE-2 F1 Score: 0.20408162765884597
BERTScore F1 (IDF) Score: 0.7302352786064148
Length Ratio: 0.4318975552968568
Project 1 was administered orally to cynomolgus monkeys at doses of 0 (vehicle), 10, 30, 100, and
800 mg/kg daily for 4 weeks, with a subsequent 4-week recovery period for the 800 mg/kg group to
assess reversibility of observed toxicities. No deaths or moribundity-related sacrifices occurred.
In the 10, 30, and 100 mg/kg groups, no significant toxicological changes were observed. However, in
the 800 mg/kg group, adverse effects such as vomiting, salivation, decreased body weight,
alterations in blood parameters (low erythrocyte count, hematocrit value, and hemoglobin
concentration), high triglycerides, high glucose, high relative liver weight, and low adrenal
weights were noted. Toxicokinetic analysis showed Tmax between 0.5 and 4 hours, with Cmax and
AUC0-24h increasing dose-dependently, indicating no significant gender differences or marked
differences after repeated do

In [17]:
######################
# ABSTRACTIVE + CHAIN (FACTS AND FACT CHECKER)
######################
checker_chain = LLMSummarizationCheckerChain(llm=llm,
                                             verbose=True,
                                             max_checks=2
                                             )

sum_abch = checker_chain.run(content)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abch)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_abch)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abch)/len(content)}")
print(sum_abch)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_summarization_checker/base.py:107: UserWarning: Directly instantiating an LLMSummarizationCheckerChain with an llm is deprecated. Please instantiate with sequential_chain argument or using the from_llm class method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMSummarizationCheckerChain chain...


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given some text, extract a list of facts from the text.

Format your output as a bulleted list.

Text:
"""
Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period. Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and 100 mg/kg groups, no toxicologically significant changes were noted in any examination during the dosing period. In the 800 mg/kg group, vomiting was

In [18]:
# ######################
# # AGENT: FACT EXTRACTOR + SUMMARIZER
# ######################
# template='''Extract all facts out of this text. Don't include opinions.
# Ensure the facts are structured to make sense to a clinical trial investigator.
# Then create a condensed summary that is readable and coherent.
# You have access to the following tools:

# {tools}

# Use the following format:

# Content: the text you are extracting key information out of
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# Thought: I now know the final answer
# Final Answer: the condensed summary

# Begin!

# Content: {prompt}
# Thought:{agent_scratchpad}'''

# tools = [PubmedQueryRun()]

# llm_prompt = ChatPromptTemplate.from_messages([
#         ("system", template),
#         ("user", "{prompt}")
#     ])


# agent = create_react_agent(llm, tools, llm_prompt)
# # Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_only_outputs=True)

# output_parser = StrOutputParser()

# chain = llm_prompt | llm | output_parser

# sum_abag = agent_executor.invoke({"prompt":content})

# # Calculate metrics
# rouge2_f1_score = calculate_rouge2_f1(target, sum_abag['output'])
# bertscore_f1_idf = calculate_bertscore_idf(target, sum_abag['output'])

# print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
# print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
# print(f"Length Ratio: {len(sum_abag['output'])/len(content)}")
# print(sum_abag['output'])

ImportError: Could not import xmltodict python package. Please install it with `pip install xmltodict`.

In [22]:
######################
# AGENT: FACT EXTRACTOR + SUMMARIZER
######################
template='''Take on the persona of a clinical trial investigator for the pharmaceutical project listed in the text.
I have a list of steps for you to perform a task. You can use the tools for all the steps.

{tools}

1) Extract all the sentences that you think are facts out of this text. Search in PubMed to see if you can validate any of the sentences.

2) Output a numbered list of all the sentences and whether you think they are facts or not.

3) Try to prioritize each of the sentences based on your knowledge as a clinical trial investigator.

4) Try to condense the sentences into fewer sentences and fewer overall words, while retaining all the factual content and all numerical information.

5) Output a second numbered list of all the condensed sentences and which fact from the original numbered list is included in each of the condensed sentences. You can just display the original sentence number.
If any of the facts from the original list are not included please redo steps 4 and 5 until all are included.

6) Take the condensed list and generate a passage that is coherent, readable and reduces redundancies, while retaining all the factual content and all numerical information.

Use the following format:

Content: the text you are extracting key information out of
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
Thought: I now know the final answer
Final Answer: the generated passage

Begin!

Content: {prompt}
Thought:{agent_scratchpad}'''

tools = [PubmedQueryRun()]

llm_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{prompt}")
    ])


agent = create_react_agent(llm, tools, llm_prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_only_outputs=False)

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

sum_abag = agent_executor.invoke({"prompt":content})

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abag['output'])
# bertscore_f1_idf = calculate_bertscore_idf(target, sum_abag['output'])

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abag['output'])/len(content)}")
print(sum_abag['output'])



> Entering new AgentExecutor chain...
Thought: I need to extract all the sentences that seem to be factual from the text and then validate these facts using PubMed.

Action: Extract all the sentences that seem to be factual from the text.

1) Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its toxicity. - Fact
2) Three males and three females were added to the 800 mg/kg group in order to assess the reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period. - Fact
3) Animals in the control group received 0.5 w/v% methylcellulose solution. - Fact
4) No animal died or was sacrificed due to moribundity in any group during the dosing or recovery period. - Fact
5) In the 10, 30, and 100 mg/kg groups, no toxicologically significant changes were noted in any examination during the dosing period. - Fact
6) In th

In [ ]:
sum_abag['output']